Dataset Evaluation
===============
This notebook produces the data analysis graphics of the pieces generated by a model.

In [19]:
from music21 import corpus, converter, instrument, note, chord, stream, pitch, key
from preprocessing import read_mxl, get_notes
import pandas as pd
from fractions import Fraction
import os
from scipy.special import softmax
import matplotlib.pyplot as plt
from statistics import mean
from PIL import Image
import pickle

In [20]:
grid_search_dataset = 'example_models/notes_slurs_dynamics_spanners_articulations_text-expressions'

In [21]:
def items_in_pieces(pieces, all_pitches, all_quarterLengths):
    pitches_in_pieces = {}
    qls_in_pieces = {}
    for k,p in pieces.items():
        pitches_in_p = []
        qls_in_p = []
        for n in p:
            pitches_in_p.append(n[0])
            qls_in_p.append(n[1])
        pitches_in_pieces.update({k:pitches_in_p})
        qls_in_pieces.update({k:qls_in_p})
    return frequencies_of_items(pitches_in_pieces, all_pitches), frequencies_of_items(qls_in_pieces, all_quarterLengths)

def frequencies_of_items(num_in_pieces, all_items):
    df_item_frequencies = pd.DataFrame(columns = ["file"] + list(all_items))
    for k,p in num_in_pieces.items():
        df_row = [k]
        for pi in df_item_frequencies.iteritems():
            if pi[0] == "file":
                continue
            freq_item = p.count(pi[0])
            df_row.append(freq_item)
        df_item_frequencies = df_item_frequencies.append(pd.Series(df_row, index=df_item_frequencies.columns), ignore_index=True)

    return df_item_frequencies

In [22]:
def sum_df_frequencies(df_pitch_frequencies):
    return df_pitch_frequencies.drop("file", axis=1).sum()

In [23]:
def concat_generation_types(gt_frequencies):
    return pd.concat(gt_frequencies, axis=1, sort=False)

In [24]:
plt.rcParams.update({'font.size': 40})
def pitches_per_piece(df_pitch_frequencies):
    return df_pitch_frequencies.astype(bool).sum(axis=1)
def produce_graphs(dataset):
    pitch_freq_series = []
    so_pitch_freq_series = []
    ql_freq_series = []
    gen_techs = []
    counted_acc_per_gen_tech = {}
    counted_pitches_per_gen_tech = {}
    counted_pitches_per_piece = []
    counted_accidentals = []
    mean_pitches_per_piece = []
    mean_accidentals = []
    for i, piece_folder in enumerate(os.listdir(dataset)+['abrsm_grade_5_violin/data']):
        full_folder_name = dataset+"/"+piece_folder if piece_folder != 'abrsm_grade_5_violin/data' else piece_folder
        if not os.path.isdir(full_folder_name):
            continue
        gen_techs.append(piece_folder)
        parts_paths, time_signatures, key_signatures, files = read_mxl(full_folder_name)
        accidentals = get_accidentals(parts_paths, key_signatures)
        mean_accidentals.append(mean(accidentals))
        counted_accidentals.append(accidentals)
        counted_acc_per_gen_tech[piece_folder] = {name:acc for name,acc in zip(files, accidentals)}
        pieces, all_pitches, all_quarterLengths = get_notes(parts_paths, single_octave=False)
        so_pieces, so_all_pitches, _ = get_notes(parts_paths, single_octave=True)
        pieces = {f:p for f, p in zip(files, pieces)}
        so_pieces = {f:p for f, p in zip(files, so_pieces)}
        df_pitch_frequencies, df_ql_frequencies = items_in_pieces(pieces, all_pitches, all_quarterLengths)
        df_so_pitch_frequencies, _ = items_in_pieces(so_pieces, so_all_pitches, all_quarterLengths)
        pitches_per_piece = df_pitch_frequencies.astype(bool).sum(axis=1)
        counted_pitches_per_gen_tech[piece_folder] = {name:acc for name,acc in zip(files, pitches_per_piece)}
        counted_pitches_per_piece.append(pitches_per_piece)
        mean_pitches_per_piece.append(pitches_per_piece.mean())
        pitch_series = sum_df_frequencies(df_pitch_frequencies)
        pitch_series = pitch_series.div(pitch_series.sum())
        pitch_freq_series.append(pitch_series)
        so_pitch_series = sum_df_frequencies(df_so_pitch_frequencies)
        so_pitch_series = so_pitch_series.div(so_pitch_series.sum())
        so_pitch_freq_series.append(so_pitch_series)
        ql_series = sum_df_frequencies(df_ql_frequencies)
        ql_series = ql_series.div(ql_series.sum())
        ql_freq_series.append(ql_series)
    

    plot_freq_dist(pitch_freq_series, gen_techs)
    plt.title("Pitch Frequency Analysis")
    plt.savefig(dataset+"/pitch_frequency_analysis.png")
    plt.clf()
    
    plt.bar(gen_techs, mean_pitches_per_piece, align='center')
    plt.title("Mean Number of Pitches Per Piece")
    plt.savefig(dataset+"/mean_pitches_per_piece.png")
    plt.clf()
    
    plt.bar(gen_techs, mean_accidentals, align='center')
    plt.title("Mean Number of Accidentals Per Piece")
    plt.savefig(dataset+"/mean_accidentals_per_piece.png")
    plt.clf()
    
    useful_plots = []
    
    plot_freq_dist(so_pitch_freq_series, gen_techs)
    plt.title("Pitch Frequency Analysis Single Octave")
    plt.savefig(dataset+"/pitch_frequency_analysis_single_octave.png")
    useful_plots.append(dataset+"/pitch_frequency_analysis_single_octave.png")
    plt.clf()
    
    plot_freq_dist(ql_freq_series, gen_techs)
    plt.title("Note Length Frequency Analysis")
    plt.savefig(dataset+"/ql_frequency_analysis.png")
    useful_plots.append(dataset+"/ql_frequency_analysis.png")
    plt.clf()
    
    ls = dict(linewidth=4.0, color='black')
    ml = dict(linewidth=4.0, color='orange')
    meanl = dict(linewidth=4.0, color='blue')
    fig, ax = plt.subplots(figsize=(30,30))
    ax.set_title('Box Plot Showing The Distribution of Accidentals Accross The Generated Pieces')
    ax.boxplot(counted_accidentals, labels=gen_techs, boxprops=ls , whiskerprops=ls, capprops=ls, flierprops=ls, medianprops=ml, meanprops=meanl, meanline=True, showmeans=True)
    plt.savefig(dataset+"/accidentals_per_piece.png")
    useful_plots.append(dataset+"/accidentals_per_piece.png")
    plt.clf()
    
    fig, ax = plt.subplots(figsize=(30,30))
    ax.set_title('Box Plot Showing The Distribution of Pitches Accross The Generated Pieces')
    ax.boxplot(counted_pitches_per_piece, labels=gen_techs, boxprops=ls, whiskerprops=ls, capprops=ls, flierprops=ls, medianprops=ml, meanprops=meanl, meanline=True, showmeans=True)
    plt.savefig(dataset+"/pitches_per_piece.png")
    useful_plots.append(dataset+"/pitches_per_piece.png")
    plt.clf()
    
    useful_plots_opened = [Image.open(p) for p in useful_plots]
    widths, heights = zip(*(i.size for i in useful_plots_opened))
    width = max(widths)
    height = max(heights)
    useful_plots_image = Image.new('RGB', (width*2, height*2))
    
    useful_plots_image.paste(useful_plots_opened[0], (0,0))
    useful_plots_image.paste(useful_plots_opened[1], (width,0))
    useful_plots_image.paste(useful_plots_opened[2], (0,height))
    useful_plots_image.paste(useful_plots_opened[3], (width,height))
    useful_plots_image.save(dataset+'/useful_plots.png')
    
    return counted_acc_per_gen_tech, counted_accidentals[-1], counted_pitches_per_gen_tech, counted_pitches_per_piece[-1]
    
def plot_freq_dist(freq_series, gen_techs):
    transposed_df = concat_generation_types(freq_series)
    column_names = {i:g for i,g in enumerate(gen_techs)}
    transposed_df = transposed_df.rename(columns = {i:g for i,g in enumerate(gen_techs)})
    width = 30
    if len(transposed_df) > 20:
        width = 100
    transposed_df.plot.bar(rot=0, figsize=(width,30))
    

def get_accidentals(parts_paths, key_signatures):
    pieces, _, _ = get_notes(parts_paths, single_octave=True)
    counted_accidentals = []
    for p, k in zip(pieces, key_signatures):
        key_object = key.KeySignature(k).getScale("major")
        notes_in_key = list(set([str(p)[:-1] for p in key_object.getPitches()]))
        all_pitches = [n[0] for n in p]
        all_accidentals = [p for p in all_pitches if p not in notes_in_key+["Rest"]]
        counted_accidentals.append(len(all_accidentals))
    return counted_accidentals

for i, piece_folder in enumerate(os.listdir(grid_search_dataset)):
    full_folder_name = "{}/test_results".format(grid_search_dataset, piece_folder)
    print(full_folder_name)
    if os.path.isdir(full_folder_name):
        for loss_val_loss in ["loss","val_loss"]:
            print("{}/{}/generated_scores".format(full_folder_name, loss_val_loss))
            produce_graphs("{}/{}/generated_scores".format(full_folder_name, loss_val_loss))

example_models/notes_slurs_dynamics_spanners_articulations_text-expressions/test_results
example_models/notes_slurs_dynamics_spanners_articulations_text-expressions/test_results/loss/generated_scores
Parsing file 0: end_seed_piece_18-1587300605.xml
Parsing file 1: end_seed_piece_31-1587302619.xml
Parsing file 2: end_seed_piece_15-1587300139.xml
Parsing file 3: end_seed_piece_0-1587297811.xml
Parsing file 4: end_seed_piece_38-1587303698.xml
Parsing file 5: end_seed_piece_46-1587304928.xml
Parsing file 6: end_seed_piece_19-1587300765.xml
Parsing file 7: end_seed_piece_27-1587301997.xml
Parsing file 8: end_seed_piece_39-1587303852.xml
Parsing file 9: end_seed_piece_20-1587300917.xml
Parsing file 10: end_seed_piece_44-1587304620.xml
Parsing file 11: end_seed_piece_42-1587304319.xml
Parsing file 12: end_seed_piece_26-1587301842.xml
Parsing file 13: end_seed_piece_11-1587299518.xml
Parsing file 14: end_seed_piece_36-1587303389.xml
Parsing file 15: end_seed_piece_17-1587300449.xml
Parsing fil

Parsing file 32: end_seed_piece_24-1587317208.xml
Parsing file 33: end_seed_piece_16-1587315979.xml
Parsing file 34: end_seed_piece_17-1587316131.xml
Parsing file 35: end_seed_piece_9-1587314905.xml
Parsing file 36: end_seed_piece_41-1587319793.xml
Parsing file 37: end_seed_piece_11-1587315212.xml
Parsing file 38: end_seed_piece_31-1587318273.xml
Parsing file 39: end_seed_piece_1-1587313675.xml
Parsing file 40: end_seed_piece_6-1587314441.xml
Parsing file 41: end_seed_piece_45-1587320404.xml
Parsing file 42: end_seed_piece_47-1587320715.xml
Parsing file 43: end_seed_piece_0-1587313516.xml
Parsing file 44: end_seed_piece_14-1587315677.xml
Parsing file 45: end_seed_piece_3-1587313984.xml
Parsing file 46: end_seed_piece_36-1587319035.xml
Parsing file 47: end_seed_piece_37-1587319186.xml
Parsing file 48: end_seed_piece_33-1587318574.xml
Parsing file 49: end_seed_piece_18-1587316283.xml
Parsing file 0: artificial_13-1587323167.xml
Parsing file 1: artificial_6-1587322091.xml
Parsing file 2: 

Parsing file 22: artificial_25-1587309411.xml
Parsing file 23: artificial_45-1587312497.xml
Parsing file 24: artificial_32-1587310485.xml
Parsing file 25: artificial_21-1587308795.xml
Parsing file 26: artificial_30-1587310179.xml
Parsing file 27: artificial_36-1587311101.xml
Parsing file 28: artificial_42-1587312032.xml
Parsing file 29: artificial_6-1587306476.xml
Parsing file 30: artificial_38-1587311411.xml
Parsing file 31: artificial_11-1587307242.xml
Parsing file 32: artificial_24-1587309259.xml
Parsing file 33: artificial_13-1587307555.xml
Parsing file 34: artificial_2-1587305861.xml
Parsing file 35: artificial_9-1587306933.xml
Parsing file 36: artificial_23-1587309105.xml
Parsing file 37: artificial_4-1587306166.xml
Parsing file 38: artificial_34-1587310797.xml
Parsing file 39: artificial_46-1587312647.xml
Parsing file 40: artificial_3-1587306010.xml
Parsing file 41: artificial_31-1587310334.xml
Parsing file 42: artificial_43-1587312186.xml
Parsing file 43: artificial_39-15873115

Parsing file 23: ABRSM_SSRTFV_8.mxl
Parsing file 24: ABRSM_SSRTFV_9.mxl
Parsing file 25: IYSR_6.mxl
Parsing file 26: IYSR_4.mxl
Parsing file 27: IYSR_5.mxl
Parsing file 28: ABRSM_SSRTFV_7.mxl
Parsing file 29: ABRSM_SSRTFV_1.mxl
Parsing file 30: ABRSM_SSRTFV_2.mxl
Parsing file 31: ABRSM_SSRTFV_3.mxl
Parsing file 32: ABRSM_y2012_g5_14.mxl
Parsing file 33: ABRSM_y2012_g5_1.mxl
Parsing file 34: IYSR_21.mxl
Parsing file 35: ABRSM_y2012_g5_15.mxl
Parsing file 36: ABRSM_y2012_g5_17.mxl
Parsing file 37: ABRSM_y2012_g5_2.mxl
Parsing file 38: ABRSM_y2012_g5_3.mxl
Parsing file 39: ABRSM_y2012_g5_16.mxl
Parsing file 40: ABRSM_y2012_g5_12.mxl
Parsing file 41: ABRSM_y2012_g5_7.mxl
Parsing file 42: ABRSM_y2012_g5_6.mxl
Parsing file 43: ABRSM_y2012_g5_11.mxl
Parsing file 44: IYSR_19.mxl
Parsing file 45: IYSR_18.mxl
Parsing file 46: ABRSM_y2012_g5_5.mxl
Parsing file 47: ABRSM_y2012_g5_10.mxl
example_models/notes_slurs_dynamics_spanners_articulations_text-expressions/test_results
example_models/notes_sl

/usr/local/lib/python3.7/site-packages/pandas/plotting/_matplotlib/core.py:338: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig = self.plt.figure(figsize=self.figsize)
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:85: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To con

...

<Figure size 7200x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>